In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(harmony)
    library(stringr)
    library(ggplot2)
    library(tidyr)
    library(Matrix)
    library(Azimuth)
    library(dplyr)
    library(RColorBrewer)
    library(UCell)
    library(Nebulosa)
    library(SeuratDisk)
    library(ggsci)
    library(DoubletFinder)
    library(scRepertoire)    
    library(scCustomize)
    library(scater)
    library(ggpubr)
    library(ggbeeswarm)
    library(UpSetR)
    library(SCP)

    source('helper_functions.R')
})

set.seed(123)

In [99]:
prepost <- readRDS("../ProcessedData/seurat/filtered_prepost_csf.rds")

In [119]:
samples <- c(
    'P1D0','P1D7','P1D21',  
    'P2D0','P2D7','P2D21',
    'P3D0','P3D7','P3D21',
    'P4D0','P4D7','P4D21',
    'P5D0','P5D0_run2','P5D7','P5D21',
    'P6D0','P6D7','P6D21', 
    'P7D0','P7D7','P7D21', 
    'P8D0','P8D7','P8D21'   
)


contig_list <- list()

for (sample in samples) {
    contig.path = Sys.glob(paste0("../RawData/cellranger_out/",sample,"/*/*/*/filtered_contig_annotations.csv"))
    contig_list[[sample]] <- read.csv(contig.path)
    if (length(contig_list[[sample]]) == 32 && colnames(contig_list[[sample]])[[1]] == 'sample') {
        contig_list[[sample]] = contig_list[[sample]][2:32] 
    }
}

In [120]:
combined.TCR <- combineTCR(
    contig_list, samples = names(contig_list), removeNA = TRUE, removeMulti = TRUE, filterMulti = TRUE, filterNonproductive = TRUE
)

In [ ]:
total_first_dim <- sum(sapply(combined.TCR, function(x) dim(x)[1]))
print(paste0("Number of T Cells pre filtering: ", total_first_dim))

In [94]:
# Function to swap parts of the barcode
swap_barcode <- function(barcode) {
  parts <- strsplit(barcode, "_")[[1]]
  paste(parts[2], parts[1], sep = "_")
}

# Apply the swap to all dataframes in combined.TCR
combined.TCR <- lapply(combined.TCR, function(df) {
  df$barcode <- sapply(df$barcode, swap_barcode)
  return(df)
})

In [95]:
Patient <- sub("^(P\\d+).*", "\\1", names(combined.TCR))
Day <- sub(".*?(D\\d+).*", "\\1", names(combined.TCR))

In [96]:
combined.TCR <- addVariable(combined.TCR,  variable.name = "Day",  variables = Day)
combined.TCR <- addVariable(combined.TCR,  variable.name = "Patient",  variables = Patient)

In [101]:
prepost <- combineExpression(combined.TCR, prepost, cloneCall="aa", 
    group.by = "sample", proportion = FALSE, 
    cloneSize=c(Single=1, Small=5, Medium=20, Large=100, Hyperexpanded=500)
)

In [52]:
saveRDS(prepost, "../ProcessedData/seurat/filtered_prepost_csf.rds")